# 数值数据 处理

数据探索
* 在处理之前，先进行探索性数据分析（EDA）
* 查看数据的分布情况
* 检查异常值和缺失值

选择合适的处理方法
* 对于正态分布数据，使用StandardScaler
* 对于有异常值的数据，使用RobustScaler
* 对于需要限定范围的数据，使用MinMaxScaler

注意事项
* 在训练集上fit，在测试集上transform
* 保存转换器的参数，用于后续预测
* 处理缺失值时要考虑业务含义
* 处理异常值时要谨慎，不要盲目删除

## 标准化/归一化

In [ ]:
import pandas as pd
import numpy as np

# 检查缺失值
missing_values = df.isnull().sum()

# 填充缺失值的几种方法
# 使用均值填充
df['列名'].fillna(df['列名'].mean())

# 使用中位数填充
df['列名'].fillna(df['列名'].median())

# 使用众数填充
df['列名'].fillna(df['列名'].mode()[0])

# 使用前向填充
df['列名'].fillna(method='ffill')

# 使用后向填充
df['列名'].fillna(method='bfill')

## 异常值

In [ ]:
# 使用IQR方法检测异常值
def detect_outliers_iqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (data < lower_bound) | (data > upper_bound)

# 使用Z-score方法检测异常值
def detect_outliers_zscore(data, threshold=3):
    z_scores = (data - data.mean()) / data.std()
    return abs(z_scores) > threshold

# 处理异常值
# 方法1：删除异常值
df = df[~detect_outliers_iqr(df['列名'])]

# 方法2：将异常值替换为上下限
def clip_outliers(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data.clip(lower=lower_bound, upper=upper_bound)

## 特征变换

In [ ]:
from sklearn.preprocessing import PowerTransformer, FunctionTransformer
import numpy as np

# 对数变换 - 处理偏斜数据
df['log_列名'] = np.log1p(df['列名'])  # log1p = log(1+x)

# Box-Cox变换 - 使数据更接近正态分布
pt = PowerTransformer(method='box-cox')
df['boxcox_列名'] = pt.fit_transform(df[['列名']])

# 平方根变换
df['sqrt_列名'] = np.sqrt(df['列名'])

# 立方根变换
df['cbrt_列名'] = np.cbrt(df['列名'])

## 特征分箱

In [ ]:
# 等宽分箱
df['等宽分箱'] = pd.qcut(df['列名'], q=10, labels=False)

# 等频分箱
df['等频分箱'] = pd.cut(df['列名'], bins=10, labels=False)

# 自定义分箱
bins = [0, 18, 35, 50, 65, 100]
labels = ['青少年', '青年', '中年', '中老年', '老年']
df['年龄分组'] = pd.cut(df['年龄'], bins=bins, labels=labels)

## 特征交互

In [ ]:
# 创建多项式特征
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(df[['特征1', '特征2']])

# 基本数学运算
df['特征1_乘_特征2'] = df['特征1'] * df['特征2']
df['特征1_加_特征2'] = df['特征1'] + df['特征2']
df['特征1_除_特征2'] = df['特征1'] / df['特征2']

## 数据平衡（对于分类问题）

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# 过采样
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# 欠采样
under_sampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = under_sampler.fit_resample(X, y)

# 类别数据 处理

使用情景：
* 如果类别之间有顺序关系（如：小、中、大），可以使用序数编码
* 如果类别之间没有顺序关系，通常使用独热编码
* 如果类别数量很多，可以考虑使用频率编码或目标编码
* 如果需要保持数据的简单性，可以使用标签编码

注意事项：
* 处理缺失值：在编码之前，需要先处理缺失值
* 新类别处理：在测试集中可能出现训练集中没有的类别，需要考虑如何处理
* 维度灾难：使用独热编码时，如果类别太多，可能会导致特征维度剧增

## 标签编码 (Label Encoding)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# 创建编码器
label_encoder = LabelEncoder()

# 假设我们有一列类别数据
categories = ['红色', '蓝色', '绿色', '红色', '蓝色']

# 进行编码
encoded_categories = label_encoder.fit_transform(categories)
# 结果会是: [2, 0, 1, 2, 0]

## 独热编码 (One-Hot Encoding)

In [ ]:
import pandas as pd

# 假设我们有一个DataFrame
df = pd.DataFrame({
    '颜色': ['红色', '蓝色', '绿色', '红色', '蓝色']
})

# 使用pandas的get_dummies进行独热编码
encoded_df = pd.get_dummies(df['颜色'])

# 结果是
#       红色     绿色     蓝色
# 0   True  False  False
# 1  False  False   True
# 2  False   True  False
# 3   True  False  False
# 4  False  False   True

## 序数编码 (Ordinal Encoding)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# 创建编码器
ordinal_encoder = OrdinalEncoder()

# 对于有顺序意义的类别（比如：小，中，大）
categories = [['小', '中', '大']]
ordinal_encoder.fit([['小'], ['中'], ['大']])

## 频率编码 (Frequency Encoding)

In [ ]:
def frequency_encode(data):
    # 计算每个类别的频率
    freq_encoding = data.value_counts(normalize=True).to_dict()
    return data.map(freq_encoding)

# 使用示例
df['颜色_freq'] = frequency_encode(df['颜色'])

## 目标编码 (Target Encoding)

In [ ]:
from category_encoders import TargetEncoder

# 创建编码器
target_encoder = TargetEncoder()

# 假设X是特征（类别列），y是目标变量
X_encoded = target_encoder.fit_transform(X, y)

# 时间数据处理

可能需要处理时区

* 如果需要进行时间序列预测，通常需要提取周期性特征
* 如果是做分类任务，可能需要将时间转换为分类特征
* 如果要分析趋势，可能需要使用滑动窗口特征

## 使用 Pandas 转换时间格式


In [ ]:
import pandas as pd

# 将字符串转换为datetime格式
df['时间列'] = pd.to_datetime(df['时间列'])

# 如果数据格式特殊，可以指定格式
df['时间列'] = pd.to_datetime(df['时间列'], format='%Y-%m-%d %H:%M:%S')

: 

## 提取时间特征

In [ ]:
# 假设df['时间列']已经是datetime格式
df['年'] = df['时间列'].dt.year
df['月'] = df['时间列'].dt.month
df['日'] = df['时间列'].dt.day
df['星期'] = df['时间列'].dt.dayofweek  # 0-6，0代表周一
df['小时'] = df['时间列'].dt.hour
df['分钟'] = df['时间列'].dt.minute
df['是否周末'] = df['时间列'].dt.dayofweek.isin([5,6]).astype(int)

## 时间差计算

In [ ]:
# 计算两个时间之间的差值
df['时间差'] = df['时间列'] - df['时间列'].shift(1)

# 转换时间差为具体数值（如天数、小时数等）
df['天数差'] = df['时间差'].dt.days
df['小时差'] = df['时间差'].dt.total_seconds() / 3600

## 时间周期特征

In [ ]:
# 季节编码
df['季节'] = df['时间列'].dt.quarter

# 月份的正弦和余弦转换（循环特征）
df['月份_sin'] = np.sin(2 * np.pi * df['时间列'].dt.month/12)
df['月份_cos'] = np.cos(2 * np.pi * df['时间列'].dt.month/12)

# 小时的周期性特征
df['小时_sin'] = np.sin(2 * np.pi * df['时间列'].dt.hour/24)
df['小时_cos'] = np.cos(2 * np.pi * df['时间列'].dt.hour/24)

## 时间窗口统计

In [ ]:
# 按照时间窗口进行重采样
# 每小时统计
hourly_stats = df.resample('H', on='时间列').mean()

# 每天统计
daily_stats = df.resample('D', on='时间列').mean()

# 每月统计
monthly_stats = df.resample('M', on='时间列').mean()

## 滑动窗口特征

In [ ]:
# 设置时区
df['时间列'] = df['时间列'].dt.tz_localize('UTC')
# 转换时区
df['时间列'] = df['时间列'].dt.tz_convert('Asia/Shanghai')